# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv') #открываю csv-файл при помощи стандартной билбиотеки

In [2]:
data.info() #изучаю общую информацию методом info()
#print(data.groupby('purpose').sum()) #здесь я по очереди выводила столбцы на экран, чтобы оценить их содержание "на глаз".
#print(data['purpose'].isna().value_counts()) проверка на наличие пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

1. Пропуски в данных (где-то количество 21525, где-то 19351 (столбцы days_employed (2174 пропусков), total_income (2174)). Вероятно, системная ошибка с какой-то группой клиентов.
2. Странный формат у days_employed (можно ли проработать, например, два с половиной дня?). Причина может быть в самом источнике данных, что данные сами по себе записываются по часам, а не по дням и затем пересчитываются в дни, поэтому есть дробное.
3. days_employed имеет отрицательные значения. Почему? Пока не ясно.
4. Пункт purpose можно категоризировать по целям (недвижимость, автомобиль, образование, свадьба, ремонт).
5. В gender помимо стандартных значений есть 1 XNA.
5. #Примечания для себя: debt имеет значения 0 или 1

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#исправление пропусков в total_income
#print(data['total_income'].mean(), data['total_income'].min(), data['total_income'].max()) расброс дохода значительный, поэтому 
 
#заменим пропуски в доходе средним значением, учитывая категорию человека (среднее среди сотрудников явно выше, чем среднее студента)
#функция принимает на вход значение дохода и если оно остуствет, то заменяет на среднее значение согалсно группе
def func(row):
    if pd.isna(row['total_income']):
        return data_grouped.loc[row['income_type']]
    return row['total_income']

data_grouped = data.groupby('income_type')['total_income'].mean()

data['total_income'] = data.apply(func, axis=1)
print(data['total_income'].isna().value_counts()) #проверили, что пропуски заполнены

False    21525
Name: total_income, dtype: int64


**Вывод**

1. Информация в days_employed не будет влиять на поставленные вопросы, поэтому заменяю пропуски на None.
2. Информация о ежемесячным доходе является важной для анализа, поэтому заменяем все все ячейки, где есть прпоуски, на среднее. 


<div class="alert alert-block alert-success">
<b>Комментарий от ревьюера</b>

Отлично всё заполнилось! Верная идея взять тип дохода!
</div>

### Замена типа данных

In [4]:
data['total_income'] = data['total_income'].astype('int')
data['debt'] = data['debt'].astype('str')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null object
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(6)
memory usage: 2.0+ MB


**Вывод**

days_employed  float64 - дней должно быть целое количество, но они нам все еще не нужны);
debt                21525 non-null int64 - имеет значения 0 или 1, поэтому переводим в тип object;
total_income        19351 non-null float64 - "округлим" (изменим тип), ведь если было выпито 100 бутылок водки, то что сделают еще пол бутылки.


### Обработка дубликатов

In [5]:
#print(data['children'].unique()) #смотрю по столбцам, чтобы выявить неявные дубликаты
data['children'] = data['children'].replace(-1, 1) #возраст ребенка -1 меняем на 1

#print(data['education'].unique()) #есть дубли, написанны "капсом". Избмавимся от них в цикле (точно можно оптимизировать, но как?)
data['education'] = data['education'].str.lower()

data=data.drop_duplicates().reset_index(drop=True)
#print(data.duplicated().sum())
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Вывод**

1. print(data.duplicated().sum()) - было обнаружено 54 явных дубликата и убрано.
2. Исправлена ошибка (дубликат со знаком минус) в возрасте детей. Сделана по ошибке.
3. Исправлены строки в образовании, приведены к порядку. Не было изначально задано строго формата внесения данных в этот пунтк, поэтому, вероятно, образовалась данная проблема. Для поиска дубликатов использован метод .unique, а дла исправления перевод всех значений к стандартному типу написания с маленькой буквы. 


### Лемматизация

In [6]:
# pymystem3 импортируется так:
from pymystem3 import Mystem
m = Mystem()

data['lemmas']=data['purpose'].apply(m.lemmatize)
print(data['lemmas'].value_counts()) #смотрю все значения, чтобы понять, как эффективнее написать функцию для разбивки по категориям

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод**

Применим лемматизацию к purpose. Заметные очевидные повторения, которые нужно разбить по категориями для удобства анализа в дальнейшем.
Описание процесса лематизации: после срабатывания кода выше мы получаем "инфинитивы" (то есть начальные формы) слов. Это нужно для того, чтобы впоследсвтии можно было провести категоризацию без учета окончаний/суффиксов и тд.

### Категоризация данных

In [7]:
#Создадим функцию, чтобы категорезировать наши данные по total_income 5 категорий

def total_income_group(income):
    if income <= 30000:
        return 'E'
    if income >= 30001 and income <= 50000 :
        return 'D'
    if income >= 50001 and income <= 200000 :
        return 'C'
    if income >= 200001 and income <= 1000000 :
        return 'B'
    if income >= 1000001:
        return 'A'
    else:
        print('Проверить вводимые данные')
    
data['category_by_income'] = data['total_income'].apply(total_income_group) 
display(data['category_by_income'].value_counts())


#Теперь напишем функцию для категоризации данных по цели запроса кредита 'purpose'
def purpose_group(goal):
    if 'свадьба' in goal:
        return 'проведение свадьбы'
    if 'автомобиль' in goal:
        return 'операции с автомобилем'
    if  'недвижимость' in goal:
        return 'операции с недвижимостью'
    if 'жилье'  in goal:
        return 'операции с недвижимостью'
    if 'образование' in goal:
        return 'получение образования'
        
data['category_by_purpose'] = data['lemmas'].apply(purpose_group)
display(data['category_by_purpose'].value_counts())


C    15514
B     5543
D      350
A       25
E       22
Name: category_by_income, dtype: int64

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: category_by_purpose, dtype: int64

**Вывод**

1. Функция по категоризации по доходу total_income_group(income) "потеряла" менее 100 клиентов при работе. Вероятно, ввиду недостатончой очистки данных на предыдущих этапах. Однако если бы было так, то сработал бы else. Требуется подсказка(
Получаемая разбивка получается неравномерной по распределению (в категории С около 15к результатов, а в остальных значительно меньше). 
2. Функция по категоризации по цели purpose_group(goal) потеряла уже 10к строк, что является крайне значительнйо потерей. При разнесении двух условий для недвижимости по двум разным строкам без использования or, потери были устранены.

if ('жилье' or 'недвижимость') in goal:
        return 'операции с недвижимостью' #данная запись фильтровала только по первому слову, как сделать так, чтобы работало верно?

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
#children и debt
data.pivot_table(index='debt', values='children') #люди, у которых есть дети, возвращают кредит чаще...кажется!
#более подробно. Создадим функцию, чтобы разбить людей по категориям относительно наличия детей
def children_categories(child):
    if child == 0:
        return 'детей нет'
    if child == 1:
        return 'один ребенок'
    if child ==2:
        return 'двое детей'
    if child ==3:
        return 'трое детей'
    if child ==4:
        return 'четверо детей'
    if child ==5:
        return 'пятеро детей'
    if child >=6:
        return 'шесть и более детей'
    #elif:
     #   print('Проверить вводимые данные')
data['children_categorised'] = data['children'].apply(children_categories)

#создадим сводную таблицу для ответа на поставленный вопрос
children_table=data.pivot_table(index='debt', columns='children_categorised', values='children', aggfunc='count')
children_table['debt'] = children_table.sum(axis=1) #сумма значений по каждой категории
children_table['debt']

print('Клиенты, у которых нет детей имеют задолжность в', (children_table.loc['1', 'детей нет']/children_table['debt'].loc['1'])*100, '% случаев')
print('Клиенты, у которых один ребенок', (children_table.loc['1', 'один ребенок']/children_table['debt'].loc['1'])*100, '% случаев')
print('Клиенты, у которых двое детей', (children_table.loc['1', 'двое детей']/children_table['debt'].loc['1'])*100, '% случаев')
print('Клиенты, у которых трое детей', (children_table.loc['1', 'трое детей']/children_table['debt'].loc['1'])*100, '% случаев')
print('Клиенты, у которых четверо детей', (children_table.loc['1', 'четверо детей']/children_table['debt'].loc['1'])*100, '% случаев')
print('Клиенты, у которых пятеро детей', (children_table.loc['1', 'пятеро детей']/children_table['debt'].loc['1'])*100, '% случаев')
print('Клиенты, у которых шестеро или более детей', (children_table.loc['1', 'шесть и более детей']/children_table['debt'].loc['1'])*100, '% случаев')

Клиенты, у которых нет детей имеют задолжность в 61.05686387133831 % случаев
Клиенты, у которых один ребенок 25.56002297530155 % случаев
Клиенты, у которых двое детей 11.143021252153934 % случаев
Клиенты, у которых трое детей 1.5508328546812178 % случаев
Клиенты, у которых четверо детей 0.22975301550832855 % случаев
Клиенты, у которых пятеро детей nan % случаев
Клиенты, у которых шестеро или более детей 0.4595060310166571 % случаев


**Вывод**

Можем видеть значительный 61% должников среди клиентов, не имеющих детей. Далее видим, что у клиентов, имеющих 1 ребенка процент задолжности 25%, а с двумя детьми уже в два раза ниже - 11%. Клиенты, имеющие трех детей 1.5и более детей имеют менее 2 % невозврата. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
#family_status и debt

family_and_debt=data.pivot_table(index='debt', columns='family_status', values='purpose', aggfunc='count')
display(family_and_debt)
#family_and_debt['debt'] = family_and_debt.sum(axis=1) #сумма значений по каждой категории
#family_and_debt['debt']
#print('Процент задолжников в статусе "не женат/не замужем":', (family_and_debt.loc['1', 'Не женат / не замужем']/family_and_debt['debt'].loc['1'])*100, '%')
#print('Процент задолжников в статусе "в разводе":', (family_and_debt.loc['1', 'в разводе']/family_and_debt['debt'].loc['1'])*100, '%')
#print('Процент задолжников в статусе "вдовец / вдова":', (family_and_debt.loc['1', 'вдовец / вдова']/family_and_debt['debt'].loc['1'])*100, '%')
#print('Процент задолжников в статусе "гражданский брак":', (family_and_debt.loc['1', 'гражданский брак']/family_and_debt['debt'].loc['1'])*100, '%')
#print('Процент задолжников в статусе "женат / замужем":', (family_and_debt.loc['1', 'женат / замужем']/family_and_debt['debt'].loc['1'])*100, '%')


Процент задолжников в статусе "не женат/не замужем": 15.738081562320506 %
Процент задолжников в статусе "в разводе": 4.882251579551982 %
Процент задолжников в статусе "вдовец / вдова": 3.6186099942561745 %
Процент задолжников в статусе "гражданский брак": 22.28604250430787 %
Процент задолжников в статусе "женат / замужем": 53.47501435956347 %


**Вывод**

1 - есть задолжность, 0 - нет. Тогда видим, что у клиентов со статусом "женат / замужем" чаще всего наблюдается задолжность. Самыми "порядочными" замещиками оказались вдовцы и вдовы. Таким образом, можно видеть, что существует зависимость между статусом клиента и тем, вернет ли он кредит без задолжности (если бы, например, везде было по 10% возврата, тогда мы бы не могли утверждать о наличии этой взаимосвязи).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
#total_income  debt
income_and_debt=data.pivot_table(index='debt', columns='category_by_income', values='income_type', aggfunc='count')
#display(income_and_debt)
income_and_debt['debt'] = income_and_debt.sum(axis=1)#сумма значений 0 и 1 по каждой категории 
#display(income_and_debt['debt']) 
print('Процент задолжников с доходом свыше 1 млн рублей:', (income_and_debt.loc['1', 'A']/income_and_debt['debt'].loc['1'])*100, '%')
print('Процент задолжников с доходом от 200 до 1 млн рублей:', (income_and_debt.loc['1', 'B']/income_and_debt['debt'].loc['1'])*100, '%')
print('Процент задолжников с доходом от 50 до 200 тыс рублей:', (income_and_debt.loc['1', 'C']/income_and_debt['debt'].loc['1'])*100, '%')
print('Процент задолжников с доходом от 30 до 50 тыс рублей:', (income_and_debt.loc['1', 'D']/income_and_debt['debt'].loc['1'])*100, '%')
print('Процент задолжников с доходом до 30 тыс рублей:', (income_and_debt.loc['1', 'E']/income_and_debt['debt'].loc['1'])*100, '%')



Процент задолжников с доходом свыше 1 млн рублей: 0.11487650775416428 %
Процент задолжников с доходом от 200 до 1 млн рублей: 22.171165996553704 %
Процент задолжников с доходом от 50 до 200 тыс рублей: 76.39287765651925 %
Процент задолжников с доходом от 30 до 50 тыс рублей: 1.2062033314187248 %
Процент задолжников с доходом до 30 тыс рублей: 0.11487650775416428 %


**Вывод**

Согласно полученным данным, можем наблюдать высокую зависимость невыплаты (debt=1) у клиентов категории дохода от 50 до 200 тыс рублей. При этом, клиенты с доходом свыше 1 млн рублей или с доходом до 30 тыс рублей заработком имеют задолжность гораздо реже.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
purpose_and_debt=data.pivot_table(index='debt', columns='category_by_purpose', values='purpose', aggfunc='count') #создаем сводную таблицу
purpose_and_debt['debt'] = purpose_and_debt.sum(axis=1)#сумма значений 0 и 1 по каждой категории 
#выведем процент по вовзрату кредита 

print('Процент задолжников в категории "операции с автомобилем":', purpose_and_debt.loc['1', 'операции с автомобилем']/income_and_debt['debt'].loc['1']*100, '%')
print('Процент задолжников в категории "операции с недвижимостью":', purpose_and_debt.loc['1', 'операции с недвижимостью']/income_and_debt['debt'].loc['1']*100, '%')
print('Процент задолжников в категории "получение образования":', purpose_and_debt.loc['1', 'получение образования']/income_and_debt['debt'].loc['1']*100, '%')
print('Процент задолжников в категории "проведение свадьбы":', purpose_and_debt.loc['1', 'проведение свадьбы']/income_and_debt['debt'].loc['1']*100, '%')

Процент задолжников в категории "операции с автомобилем": 23.147616312464102 %
Процент задолжников в категории "операции с недвижимостью": 44.916714531878235 %
Процент задолжников в категории "получение образования": 21.25215393452039 %
Процент задолжников в категории "проведение свадьбы": 10.683515221137277 %


**Вывод**

Видим, что цели кредита влияют на задолжность при его возврате. Так больше всего должников в атегории операций с недвижимостью, что вполне объяснимо, ведь зачастую недвижимость стоит дороже автомобилей\свадеб\образования. 
Чаще всего кредит вовзрващают люди, цель которых была проведение свадбы. 

## Шаг 4. Общий вывод

Был дан на анализ дата сет раземера  21525 с наличием пропусков по категориям days_empoyed (не пригодились при анализе) и  total_income (пропуски заполнены среднем для кажедой катеогрии дохода). 
Был исправлен тип данных  total_income на int для удобства анализа и экономии памяти (отброшены значения после запятой) и в debt на строчный тип str, так как это булевые значения 0/1.
Была проведена проверка на наличие дубликатов и их исправление в колонке education.
Произведена лемматизация столбца с целями кредита для последующей разбивки на категории для удобства анализа. Также на категории были разбиты клиенты по их уровню дохода (катеогрии А-Е).

<b>Есть ли зависимость между наличием детей и возвратом кредита в срок?</b>
Опираясь на полученные данные и рассматривая три категории клиентов (без детей, 1 ребенок, 2 и более) можно наблюдать тенденцию к уменьшению вероятности задолжности при увеличении количества детей, так как наибольший процент задолжностей у людей без детей. В данную категоию могут входить клиенты со статусом студент или люди, начинающие карьерный путь/меняющие его, а значит имеющие нестабильный или низкий заработок и низкую возможность возврата кредита, что потверждает наблюдаемое. 
<b>New!</b> При более детальном рассмтроении категорий клиентов, имеющих летей заметим следующее: клиенты с 1 ребенком показали  процент задолжности 25%, а с двумя детьми уже в два раза ниже - 11%. Клиенты, имеющие тех и более детей имеют менее 2 % невозврата. 

<b>Есть ли зависимость между семейным положением и возвратом кредита в срок?</b>
Наблюдаем, что клиенты со статусом "гражданский брак" (22%) и "женат / замужем" (53%) имеют довольно высокие проценты невозврата. Одинокие же клиенты имеют процент значительно ниже. При рассматроении стандартной схемы семьи с детьми и учитывая выводы из предыдущего вопроса, возникает противоречие. Люди, у которых есть дети возвращают долги, однако в то же время видим, что жентаые/замужем имеют высокий процент невозврата. Значит, не стоит рассматривать данные относительно модели реальной семьи, а стоит сделать вывод о том, что надежнее дать кредит свободному человеку (в статусе "не женат/не замужем" или "разведен(а)").

<b>Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</b>
А: 0.1 %
В: 22.2 %
C: 76.4 %
D: 1.2%
E: 0.1 %
Полученная статистика говорит нам о том, что люди в категориях с высоким (более 1 млн рублей) доходом практически всегда выплачивают кредит. В то же время клиенты с низкими доходами (до 50 тыс рублей) дают схожих показтель возврата. Вероятно, это связано с суммой кредита, которую берут клиенты с низким доходом, либо частота, с которой они его берут (вероятно, что клиент взял один кредитЮ чтобы погасить другой и тогда по другому кредиту статистика возрвата будет хорошей). Наименее надежными  будут являться клиенты категории С с доходом в рамках от 100к до 200к рублей. Вероятно, банк имел к таким клиентам высокое доверие в прошлом ввиду среднего размера заработка (=стабильность). Однако пришло время это пересмотреть.


<b>Как разные цели кредита влияют на его возврат в срок?</b>

Клиенты с целями "свадьба" и "образования" оказываются наиболее лояльными и имеют около 10% задолжности. Наименьшее доверие вызывают клиенты, выбирающие цель "операции с недвижимостью". Вероятно, это может быть связанно с тем, что недвижимость имеет набильшую стоимость, сравнивая с остальными категориями (за редкими исключениями). Также не стоит забывать, что кредиты на операции с недвижимостью выдаются на значительные временные сроки, так что статус клиента в это время может изменяться (семейное положение, доход), что точно нужно учитывать при принятия решения о выдаче кредита (например, оценивать стабильность работы на одном рабочем месте, возраст клиента и его будущая трудоспособностьЮ надежность поручителя и тд). 



Проанализировав зависимости выше, были собраны следующие хайлайты:
Клиенты, у которых нет детей имеют задолжность в 61 % случае
Процент задолжников в статусе "женат / замужем": 53 %
Процент задолжников в категории C: 76 %
Процент задолжников в категории "операции с недвижимостью": 45 %
Значит, клиент со средним достатком категории С, женаты, не имеющей детей и избрваший целью кредит "недвижимость" 
получить бОльший процент по кредиту или ему с бОльшей вероятностью откажут, так как со стороны банка он/она является ненадежным.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.